In [14]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import pyBigWig
from kipoiseq import Interval
import kipoiseq
from Bio.Seq import Seq
import pyfaidx

In [15]:
# extract DNA sequence
class FastaStringExtractor:
    def __init__(self, fasta_file):
        self.fasta = pyfaidx.Fasta(fasta_file)
        self._chromosome_sizes = {k: len(v) for k, v in self.fasta.items()}

    def extract(self, interval: Interval, **kwargs) -> str:
        # Truncate interval if it extends beyond the chromosome lengths.
        chromosome_length = self._chromosome_sizes[interval.chrom]
        trimmed_interval = Interval(interval.chrom,
                                    max(interval.start, 0),
                                    min(interval.end, chromosome_length),
                                    )
        # pyfaidx wants a 1-based interval
        sequence = str(self.fasta.get_seq(trimmed_interval.chrom,
                                          trimmed_interval.start + 1,
                                          trimmed_interval.stop).seq).upper()
        # Fill truncated values with N's.
        pad_upstream = 'N' * max(-interval.start, 0)
        pad_downstream = 'N' * max(interval.end - chromosome_length, 0)
        return pad_upstream + sequence + pad_downstream

    def close(self):
        return self.fasta.close()

In [16]:
def getInput(seq):
    # Extract fa file
    fasta_extractor = FastaStringExtractor(seq)
    target_interval = kipoiseq.Interval('chr1', 10000, 10100)
    seqs = fasta_extractor.extract(target_interval)
    seqs_2d = np.array(list(seqs)).reshape(-1, 1)

    # One-hot
    encoder = OneHotEncoder(categories=[['A', 'C', 'G', 'T']], sparse_output=False)
    f_input = encoder.fit_transform(seqs_2d)

    # Convert np matrix to torch matrix
    f_input = torch.tensor(f_input, dtype=torch.float32).unsqueeze(0).permute(0, 2, 1)

    return f_input

In [17]:
class SignalCNN(nn.Module):
    def __init__(self):
        super(SignalCNN, self).__init__()
        
        # layers
        self.conv1 = nn.Conv1d(in_channels=4, out_channels=16, kernel_size=25, stride=1, padding=12)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=1, kernel_size=1)
        
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.conv3(x)
        return x

In [18]:
model = SignalCNN()
dataset = model(getInput("Data/hg38.fa"))
display(dataset.shape)
display(dataset)

torch.Size([1, 1, 100])

tensor([[[0.1557, 0.1475, 0.1450, 0.1585, 0.1520, 0.1284, 0.1488, 0.1381,
          0.1510, 0.1609, 0.1619, 0.1384, 0.1754, 0.1408, 0.1634, 0.1777,
          0.1633, 0.1314, 0.1725, 0.1408, 0.1634, 0.1777, 0.1633, 0.1314,
          0.1725, 0.1408, 0.1634, 0.1777, 0.1633, 0.1314, 0.1725, 0.1408,
          0.1634, 0.1777, 0.1633, 0.1314, 0.1725, 0.1408, 0.1634, 0.1777,
          0.1633, 0.1314, 0.1725, 0.1408, 0.1634, 0.1777, 0.1633, 0.1314,
          0.1725, 0.1408, 0.1634, 0.1777, 0.1633, 0.1314, 0.1725, 0.1408,
          0.1634, 0.1777, 0.1633, 0.1314, 0.1725, 0.1408, 0.1634, 0.1777,
          0.1633, 0.1314, 0.1725, 0.1408, 0.1634, 0.1777, 0.1633, 0.1314,
          0.1725, 0.1408, 0.1634, 0.1777, 0.1633, 0.1314, 0.1725, 0.1408,
          0.1634, 0.1777, 0.1633, 0.1314, 0.1725, 0.1408, 0.1634, 0.1719,
          0.1596, 0.1503, 0.1815, 0.1353, 0.1548, 0.1561, 0.1555, 0.1315,
          0.1786, 0.1399, 0.1300, 0.1404]]], grad_fn=<ConvolutionBackward0>)

In [ ]:
train_output = trainer.SeqModelTrainer()

In [ ]:
# Load files
bw = pyBigWig.open("Data/example/ENCSR000AKO_plus.bigWig")
print(bw.chroms())

In [ ]:
print(bw.values('1', 1000000, 1300000))